In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}

# MD with openMM - python API
This jupyter notebook shows how to run a Molecular dynamics (MD) simulations using the openMM python package. First of all we import the openMM packages that we need for the simulations, plus some general one for handling the output.

In [ ]:
########################## import openMM ###############################
import openmm as mm
from openmm.app import *
from openmm.unit import *
from openmmtools import integrators as mmt

########################## File handling libraries #####################
import subprocess
from sys import stdout

########################## Initialise random seeds #####################
import time
import random
# random.seed(123456) # <- use this for reproducibility
random.seed(int(time.time()))

First of all we read the size and the atomic coordinates of the system we want to simulate.

In [41]:
pdb    = PDBFile('Coordinates/fcc.pdb')
box    = pdb.topology.getPeriodicBoxVectors()
print("Number of atoms in the system   : %i " % pdb.topology.getNumAtoms())
print("Simulation cell dimensions [nm] : %8.3f %8.3f %8.3f" % 
      (box[0][0].value_in_unit(nanometer),
       box[1][1].value_in_unit(nanometer),
       box[2][2].value_in_unit(nanometer))
     )

Number of atoms in the system   : 2048 
Simulation cell dimensions [nm] :    4.371    4.371    4.371


We can now create our __system__ by adding the dimensions of the simulation cell that was in the PDB file.

In [ ]:
system = mm.System()
system.setDefaultPeriodicBoxVectors(box[0],box[1],box[2])

We then define how the atoms interact. OpenMM has some optimised built is force fields that we can readily use, or define our own custom interactions. The class __NonbondedForce__ describes the interactions between particles as the sum of the _Coulomb_ and _van der Waals_ forces. The energy terms associated with these forces are well known terms

\begin{equation}
U_{ij}(r) = \frac{1}{4\pi\epsilon_0} \frac{q_iq_j}{r} + 4\varepsilon_{ij}\Big[ \Big(\frac{\sigma_{ij}}{r}\Big)^{12} - \Big(\frac{\sigma_{ij}}{r}\Big)^6 \Big]
\end{equation}

where $\varepsilon_{ij}$ and $\sigma_{ij}$ are parameters specific to the interacting atoms, and $q_i$ and $q_j$ are their charges.

In [ ]:
atomForceField = []
atomForceField.append({
    "type" : "Ne", 
    "mass" : 20.180 * amu,
    "sigma" :  2.72 * angstrom,
    "epsilon" : 0.390764250 * kilojoule_per_mole,
})
atomForceField.append({
    "type" : "Ar",
    "mass" : 39.948 * amu,
    "sigma" :  3.40 * angstrom,
    "epsilon" : 0.996014655 * kilojoule_per_mole,
})
atomForceField.append({
    "type" : "Kr",
    "mass" : 83.798 * amu,
    "sigma" :  3.67 * angstrom,
    "epsilon" : 1.388419150 * kilojoule_per_mole,
})
atomForceField.append({
    "type" : "Xe",
    "mass" : 131.29 * amu,
    "sigma" :  3.92 * angstrom,
    "epsilon" : 2.140037300 * kilojoule_per_mole,
})

We now create the system and add the atoms with their interactions.
In this cases all the charges are zero.
Note that if the system has more than one atom type the cross terms of the force field are computed using __mixing rules__.

\begin{eqnarray}
\sigma_{ij} &=& (\sigma_i + \sigma_j) / 2 \\ 
\epsilon_{ij} &=& \sqrt{\epsilon_i + \epsilon_j}
\end{eqnarray}

In [ ]:
charge = 0.0

force = mm.NonbondedForce()
force.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
force.setCutoffDistance(1.5*nanometer)
force.setUseDispersionCorrection(True)

for i in pdb.topology.atoms():
    iType = next(
        (t for t,d in enumerate(atomForceField) if d['type'] == i.name), None)
    mass = atomForceField[iType]["mass"]
    sigma = atomForceField[iType]["sigma"]
    epsilon = atomForceField[iType]["epsilon"]
    
    system.addParticle(mass)
    force.addParticle(charge, sigma , epsilon)

_ = system.addForce(force)

We can then set the ensemble for the simulation. 
1. NVE - microcanonical, _i.e._ constant number of atoms (N), Volume and Energy
2. NVT - canonical,  _i.e._ constant number of atoms (N), Volume and Temperature
3. NPH - isobaric-isoenthalpic, _i.e._ constant number of atoms (N), Pressure and entHalpy
4. NPT - isothermal-isobaric,  _i.e._ constant number of atoms (N), Pressure and Temperature

For the constant pressure simulations there are also various options for how to control the shape of the simulation cell. In this case we can choose between isotropic or orthorhombic deformations of the system.

We also need to define the simulation temperature and pressure, the timestep and a few other parameters.

In [ ]:
########################## Simulation parameters ##########################
minimise    = False             # <-- perform energy minimisation
NVE         = False             # <-- MD with no thermostat (VV)
NPT_iso     = True              # <-- montecarlo barostat isotropic
NPT_ort     = False             # <-- montecarlo barostat orthorhombic

timestep    = 0.002*picoseconds # <-- MD timestep
nsteps      = 20000             # <-- total number of timesteps
ntraj       = 1000              # <-- frequency of trajectory output
nthermo     = 1000              # <-- frequency of data output

temperature = 100*kelvin        # <-- temperature
pressure    = 1*atmosphere      # <-- external pressure

trel        = 1/picoseconds     # <-- thermostat relaxation time
nupdt       = 25                # <-- how often the volume is updated

According to the ensemble chosen, we add different integrators to the __system__.

In [ ]:
########################## Set integrator #################################
if NVE:
    integrator = mmt.VelocityVerletIntegrator( timestep )
else:
    integrator = mmt.LangevinIntegrator( temperature , trel , timestep )

# Barostat
if NPT_iso:
    system.addForce(mm.MonteCarloBarostat( pressure , temperature , nupdt ))
if NPT_ort:
    system.addForce(mm.MonteCarloAnisotropicBarostat(
        (pressure, pressure, pressure), temperature,  False, False, True))

The next thing we have to do is to choose the device we want to use for our simulation. In this case we select __OpenCL__.

In [ ]:
########################## Initialise GPU / CUDA / OpenCL #################
platform = mm.Platform.getPlatformByName('OpenCL')
properties = {'Precision': 'mixed'} # <-- use double for energy conservation

# platform = mm.Platform.getPlatformByName('CUDA')
# properties = {'Precision' : 'mixed' , 
#               'DeviceIndex' : '0' , 
#               'CudaCompiler' : '/usr/bin/nvcc'}

# platform = mm.Platform.getPlatformByName('CPU')
# properties = { 'Threads' : '1' }

We can now create the __simulation__ object.

In [ ]:
########################## Create simulation object #######################
simulation = mm.app.Simulation(
    pdb.topology, system, integrator, platform)

We then add the atoms' coordinates and generate their initial velocities.

In [ ]:
########################## Initialise positions and velocities ############
simulation.context.setPositions(pdb.positions)
simulation.context.setVelocitiesToTemperature(temperature , 
                                              random.randrange(99999) )

In some instances it is good to start the simulations by doing an energy minimisation, to remove unphysical close contacts between the particles.

In [ ]:
########################## Energy minimisation ############################
if minimise:
    simulation.minimizeEnergy()

We now customise the output of out simulation. We set both a screen output and a file output.

In order to avoid overwriting the output files, we set a convention for the file names and count how many simulations we have already run in the current folder.

In [ ]:
########################## Output files ################################
nn = subprocess.getoutput("ls output.*.dat 2> /dev/null | wc -l")
ftraj   = 'trajectory.' + nn.strip() + '.dcd'    # <-- trajectory output file
fthermo = 'output.' + nn.strip() + '.dat'  # <-- output data filename

########################## Initialise the outputs #########################
# Screen output
simulation.reporters.append(
    StateDataReporter( 
        sys.stdout, int(nsteps/20), totalSteps = nsteps, separator= "\t", 
        step=False, time=True, potentialEnergy=False, kineticEnergy=False, 
        totalEnergy=False, temperature=False, volume=False, density=False, 
        progress=True, remainingTime=True, speed=True, elapsedTime=False
    )
)


# File output
simulation.reporters.append(
    StateDataReporter(
        fthermo , nthermo, separator= "\t",
        step=False, time=True, potentialEnergy=True, kineticEnergy=False, 
        totalEnergy=False, temperature=True, volume=True, density=True, 
        progress=False, remainingTime=False, speed=False, elapsedTime=False
    )
)



# Trajectory
simulation.reporters.append(
    DCDReporter( ftraj , ntraj )
)

We're finally ready to run out MD simulations

In [ ]:
simulation.step( nsteps )